In [2]:
%pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/work/projects/Novozymes/data/termonet/wildtypeA.zip 

Archive:  /content/drive/MyDrive/work/projects/Novozymes/data/termonet/wildtypeA.zip
   creating: wildtypeA/
 extracting: wildtypeA/wildtypeA_A101E.resfile  
 extracting: wildtypeA/wildtypeA_A101F.resfile  
 extracting: wildtypeA/wildtypeA_A101G.resfile  
 extracting: wildtypeA/wildtypeA_A101H.resfile  
 extracting: wildtypeA/wildtypeA_A101I.resfile  
 extracting: wildtypeA/wildtypeA_A101K.resfile  
 extracting: wildtypeA/wildtypeA_A101N.resfile  
 extracting: wildtypeA/wildtypeA_A101S.resfile  
 extracting: wildtypeA/wildtypeA_A101T.resfile  
 extracting: wildtypeA/wildtypeA_A101W.resfile  
 extracting: wildtypeA/wildtypeA_A101Y.resfile  
 extracting: wildtypeA/wildtypeA_A111C.resfile  
 extracting: wildtypeA/wildtypeA_A111G.resfile  
 extracting: wildtypeA/wildtypeA_A111I.resfile  
 extracting: wildtypeA/wildtypeA_A111K.resfile  
 extracting: wildtypeA/wildtypeA_A111P.resfile  
 extracting: wildtypeA/wildtypeA_A111R.resfile  
 extracting: wildtypeA/wildtypeA_A111S.resfile  
 extracti

In [4]:
class CFG:
  TEST_CSV = '/content/drive/MyDrive/work/projects/Novozymes/data/test.csv'


In [5]:
import pandas as pd
from lightgbm import LGBMRegressor
from scipy.stats import spearmanr, pearsonr
import numpy as np
from matplotlib import pyplot as plt
from lightgbm import plot_importance, plot_tree
import Levenshtein
from scipy.stats import rankdata
from io import StringIO
import pandas as pd

def gen_mutations(name, df,
                  wild="VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQ""RVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGT""NAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKAL""GSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"):
    result = []
    for _, r in df.iterrows():
        ops = Levenshtein.editops(wild, r.protein_sequence)
        assert len(ops) <= 1
        if len(ops) > 0 and ops[0][0] == 'replace':
            idx = ops[0][1]
            result.append([ops[0][0], idx + 1, wild[idx], r.protein_sequence[idx]])
        elif len(ops) == 0:
            result.append(['same', 0, '', ''])
        elif ops[0][0] == 'insert':
            assert False, "Ups"
        elif ops[0][0] == 'delete':
            idx = ops[0][1]
            result.append(['delete', idx + 1, wild[idx], '-'])
        else:
            assert False, "Ups"

    df = pd.concat([df, pd.DataFrame(data=result, columns=['op', 'idx', 'wild', 'mutant'])], axis=1)
    df['mut'] = df[['wild', 'idx', 'mutant']].astype(str).apply(lambda v: ''.join(v), axis=1)
    df['name'] = name
    return df

def parse_score(pdb_file):
  f = open(pdb_file, 'r')
  lines = f.readlines()
  start = False
  for i,line in enumerate(lines):
    if line.startswith('#BEGIN_POSE_ENERGIES_TABLE'):
      break
  lines = lines[i+1:-2]
  table_data = ''
  for line in lines:
    table_data += line

  df = pd.read_csv(StringIO(table_data), sep=' ')
  return df

def ensamble(subissions, weights, out_file_name):
  dfe   = pd.read_csv(f'submission_0{subissions[0]}.csv')
  coeff = 1.0/len(subissions)   
  dfe.tm = 0.0*dfe.tm
  for i,subission in enumerate(subissions):
    if weights is not None:
      coeff = weights[i]
    print(f'{coeff}x{subission}')
    dfs   = pd.read_csv(f'submission_0{subission}.csv')
    dfe.tm += coeff*rankdata(dfs.tm)
  dfe.tm = dfe.tm/dfe.tm.max()
  dfe.to_csv(out_file_name, index=False)  



In [6]:
df_test = gen_mutations('wildtypeA', pd.read_csv(CFG.TEST_CSV))

df_test
#df = parse_score('/content/wildtypeA/wildtypeA_relaxed_F164L_relaxed.pdb')

,seq_id,protein_sequence,pH,data_source,op,idx,wild,mutant,mut,name
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,17,L,E,L17E,wildtypeA
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,17,L,K,L17K,wildtypeA
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,delete,17,L,-,L17-,wildtypeA
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,18,K,C,K18C,wildtypeA
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,18,K,F,K18F,wildtypeA
...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,16,A,I,A16I,wildtypeA
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,16,A,L,A16L,wildtypeA
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,16,A,N,A16N,wildtypeA
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,replace,16,A,P,A16P,wildtypeA


In [9]:
df = parse_score('/content/wildtypeA/wildtypeA_relaxed_F164L_relaxed.pdb')
df.head()

,label,fa_atr,fa_rep,fa_sol,fa_intra_rep,fa_intra_sol_xover4,lk_ball_wtd,fa_elec,pro_close,hbond_sr_bb,...,hbond_sc,dslf_fa13,coordinate_constraint,omega,fa_dun,p_aa_pp,yhh_planarity,ref,rama_prepro,total
0,weights,1.00000,0.55000,1.00000,0.00500,1.00000,1.00000,1.00000,1.25000,1.0000,...,1.0000,1.25000,0.50000,0.40000,0.70000,0.60000,0.62500,1.00000,0.45000,NaN
1,pose,-1195.76000,131.47800,773.80000,2.39855,41.46660,-42.37580,-367.83000,2.13017,-67.6216,...,-26.5253,-2.26843,6.63566,26.80600,240.53500,-39.43230,0.10947,16.51190,-4.57845,-597.56300
2,VAL:NtermProteinFull_1,-0.99436,0.33910,0.44911,0.01853,0.05373,-0.10441,-0.23825,0.00078,0.0000,...,0.0000,0.00000,0.08735,0.00036,0.12670,0.00000,0.00000,2.64269,0.00000,2.38133
3,PRO_2,-1.58292,0.57338,0.47748,0.00335,0.10606,-0.15440,0.16294,0.24484,0.0000,...,0.0000,0.00000,0.02889,0.05432,0.53473,-0.62379,0.00000,-1.64321,0.32621,-1.49211
4,VAL_3,-2.28422,0.38203,0.43661,0.01979,0.06475,-0.20808,-0.06456,0.00000,0.0000,...,0.0000,0.00000,0.03214,0.49868,0.41459,-0.15758,0.00000,2.64269,0.35836,2.13520


In [7]:

csv_list = ['338','368_constr','472_roset','482_p3','488_term','590','603_del_spec','613','614','615']
csv_dfs = {}

def norm_rank_df(df):
  df.tm = rankdata(df.tm)
  df.tm = df.tm / df.tm.max()
  return df

def norm_min_max_df(df):
  df.tm = (df.tm - df.tm.min())/(df.tm.max() - df.tm.min())
  return df

for csv_file in csv_list:
  df_other = pd.read_csv(f'/content/submission_0{csv_file}.csv')
  csv_dfs[csv_file] = df_other

df_val = pd.read_csv(f'/content/submission_v2.csv')

def checkcsv(file_name):
  df = pd.read_csv(file_name)
  print(f'val: {spearmanr(norm_rank_df(df).tm, norm_rank_df(df_val).tm).correlation:.4f}')
  for csv_score in csv_list:
    sp = spearmanr(norm_rank_df(df).tm, norm_rank_df(csv_dfs[csv_score]).tm).correlation
    p  = pearsonr(norm_min_max_df(df).tm, norm_min_max_df(csv_dfs[csv_score]).tm)[0]
    print(f'sp: {sp:.4f} p: {p:.4f} - 0.{csv_score}')

def submit(scores, file_name):
  # replacement mutations
  df_test.loc[df_test.op == 'replace', 'ddg'] = scores
  # deletion mutations
  df_test.loc[df_test['op'] == "delete", 'ddg'] = df_test[df_test["op"]=="replace"]["ddg"].quantile(q=0.25)
  # no mutations
  df_test.loc[df_test['op'] == "same", 'ddg'] = 0.  

  df = df_test.rename(columns={'ddg': 'tm'})[['seq_id', 'tm']]
  df.to_csv(file_name, index=False)



df_test_repl = df_test.loc[df_test.op == 'replace'].copy()


FileNotFoundError: ignored

In [ ]:

mutations = {}
for i,row in df_test_repl.iterrows():
  file_name = f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb'
  mutations[file_name] = parse_score(file_name)

- 0.5938 coordinate_constraint 
- 0.3209 fa_dun 
- 0.310  fa_rep 
- 0.2854 rama_prepro 
- 0.2648 omega 
- 0.256  fa_sol 
- 0.2155 hbond_lr_bb 
- 0.208  fa_intra_sol_xover4 
- 0.1919 p_aa_pp 
- 0.1656 hbond_sr_bb 
- 0.140  lk_ball_wtd 
- 0.1069 yhh_planarity 
- 0.096  hbond_bb_sc 
- 0.090  fa_intra_rep 
- 0.0157 pro_close 
- 0.010  fa_atr 
- 0.0066 dslf_fa13 
- -0.026 fa_elec 
- -0.064 hbond_sc 
- -0.185 ref 


In [ ]:
#scores = df_test_repl.apply(lambda row: -parse_score(f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb').loc[1,'total'], axis=1)
#scores = df_test_repl.apply(lambda row: -parse_score(f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb').loc[row.idx+1,'total'], axis=1)

props_all = ["fa_atr", 
             "fa_rep", 
             "fa_sol",
             "fa_intra_rep", 
             "fa_intra_sol_xover4", 
             "lk_ball_wtd", 
             "fa_elec",
             "pro_close", 
             "hbond_sr_bb", 
             "hbond_lr_bb", 
             "hbond_bb_sc", 
             "hbond_sc", 
             "dslf_fa13", 
             "coordinate_constraint", 
             "omega", 
             "fa_dun", 
             "p_aa_pp", 
             "yhh_planarity", 
             "ref", 
             "rama_prepro" ]

#https://new.rosettacommons.org/demos/latest/tutorials/scoring/scoring#score-function

weights_default = [
    1.0,
    0.55,
    1.0,
    0.005,
    1.0,
    1.0,
    1.0,
    1.25,
    1.0,
    1.0,
    1.0,
    1.0,
    1.25,
    0.5,
    0.4,
    0.7,
    0.6,
    0.625,
    1.0,
    0.45,
]

weights_ref2015 = [
    1.0,    # "fa_atr"                1.16979554
    0.55,   # "fa_rep"                0.9166202
    0.9375, # "fa_sol"                0.86421302
    0.005,  # "fa_intra_rep"          0.00485843
    1.0,    # "fa_intra_sol_xover4" unknown 1.0  0.90040041
    1.0,    # "lk_ball_wtd"         unknown 1.0  1.27891325    
    0.875,  # "fa_elec"               0.7547942
    1.25,   # "pro_close"             1.89225524
    1.17,   # "hbond_sr_bb"           0.8616921
    1.17,   # "hbond_lr_bb"           1.13859553
    1.17,   # "hbond_bb_sc"           1.57343679        
    1.1,    # "hbond_sc"              -0.02813014
    1.25,   # "dslf_fa13"             0.96473408
    0.5,    # "coordinate_constraint" unknown 0.5    0.61507348
    0.625,  # "omega"                 0.64580646
    0.7,    # "fa_dun"                0.50213331
    0.4,    # "p_aa_pp"               0.39317266
    0.625,  # "yhh_planarity"         0.58628116
    1.0,    # "ref"                   0.79924245
    0.25    # "rama_prepro"           0.30142596
]

props_best = ['coordinate_constraint', 
              'fa_dun', 
              'fa_rep', 
              'rama_prepro', 
              'omega', 
              'fa_sol', 
              'hbond_lr_bb']


def score(mut_file, weights):
  mut_df = mutations[mut_file]
  val = 0
  for i, prop in enumerate(props_all):
    val += ((mut_df.loc[1,prop]) / mut_df.loc[0,prop])*weights[i] # sp: 0.7769 k: 0.480 
  return -val
  
weights = weights_default

scores = df_test_repl.apply(lambda row: score(f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb', weights), axis=1)
submit(scores, 'submission.csv')
checkcsv('submission.csv')

val: 0.4748
sp: 0.5797 p: 0.5797 - 0.338
sp: 0.6129 p: 0.6129 - 0.368_constr
sp: 1.0000 p: 1.0000 - 0.472_roset
sp: 0.3484 p: 0.3484 - 0.482_p3
sp: 0.4963 p: 0.4963 - 0.488_term
sp: 0.8192 p: 0.8192 - 0.590
sp: 0.7909 p: 0.7909 - 0.603_del_spec
sp: 0.7461 p: 0.7461 - 0.613
sp: 0.7583 p: 0.7583 - 0.614
sp: 0.7695 p: 0.7695 - 0.615


# Train weights

In [ ]:
import numpy as np
from scipy.optimize import fmin, fmin_bfgs
import math

weights_ref2023 = [
    1.0,    # "fa_atr"                1.16979554
    0.55,   # "fa_rep"                0.9166202
    0.9375, # "fa_sol"                0.86421302
    0.005,  # "fa_intra_rep"          0.00485843
    1.0,    # "fa_intra_sol_xover4" unknown 1.0  0.90040041
    1.0,    # "lk_ball_wtd"         unknown 1.0  1.27891325    
    0.875,  # "fa_elec"               0.7547942
    1.25,   # "pro_close"             1.89225524
    1.17,   # "hbond_sr_bb"           0.8616921
    1.17,   # "hbond_lr_bb"           1.13859553
    1.17,   # "hbond_bb_sc"           1.57343679        
    1.1,    # "hbond_sc"              -0.02813014
    1.25,   # "dslf_fa13"             0.96473408
    0.5,    # "coordinate_constraint" unknown 0.5    0.61507348
    0.625,  # "omega"                 0.64580646
    0.7,    # "fa_dun"                0.50213331
    0.4,    # "p_aa_pp"               0.39317266
    0.625,  # "yhh_planarity"         0.58628116
    1.0,    # "ref"                   0.79924245
    0.25    # "rama_prepro"           0.30142596
]

def score(mut_file, weights):
  mut_df = mutations[mut_file]
  val = 0
  for i, prop in enumerate(props_all):
    val += ((mut_df.loc[1,prop]) / mut_df.loc[0,prop])*weights[i] # sp: 0.7769 k: 0.480 
  return -val

def calc_score(weights):
  df_test_repl = df_test.loc[df_test.op == 'replace'].copy()
  scores = df_test_repl.apply(lambda row: score(f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb', weights), axis=1)
  df_test.loc[df_test.op == 'replace', 'ddg'] = scores
  df_test.loc[df_test['op'] == "delete", 'ddg'] = df_test[df_test["op"]=="replace"]["ddg"].quantile(q=0.25)
  df_test.loc[df_test['op'] == "same", 'ddg'] = 0.  

  df = df_test.rename(columns={'ddg': 'tm'})[['seq_id', 'tm']]
  sp = spearmanr(norm_rank_df(df).tm, norm_rank_df(df_val).tm).correlation
  np.save('best_weights', weights)
  print(sp)
  return 1-sp

weights_start = np.array(weights_default)
#weights_start = np.load('best_weights_5150.npy')

print('started from:', 1.0-calc_score(weights_start))

new_weights = fmin(calc_score, weights_start, disp=True)
print('new weights:', new_weights)
print('new score:', 1.0-calc_score(new_weights))

0.47478306727105807
started from: 0.474783067271058
0.47478306727105807
0.4758863602272325
0.4755370621620954
0.47534003403222436
0.47475645902274166
0.4747430109801858
0.47550182051815154
0.4715725424896898
0.4747579007690632
0.4747906330227862
0.47484914556161273
0.4753054526192678
0.4730136345591483
0.47476748308658845
0.4752182030515853
0.474955455887963
0.47433349352855136
0.4746593709029981
0.4747818706557757
0.4740257157076012
0.474811603257199
0.47764292569486766
0.479670331920997
0.4774023880866965
0.4762254311179294
0.47657143059047524
0.47661078992341005
0.4767084981307303
0.47686987976609085
0.4770463143263166
0.4772190257987368
0.4775589721574311
0.47789108115578927
0.4781024584196671
0.478581900567901
0.4787109411342634
0.478931418994739
0.4790614298358771
0.4792542557193603
0.4795873640324083
0.4799253946108896
0.48162388281274715
0.4801846287990257
0.47909219335424963
0.4804838441159034
0.480428678528568
0.4807163077738503
0.4811511121577113
0.4813371119531599
0.4815801

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0.510678103302468
0.5112541142684528
0.5112192381857801
0.5099511770277964
0.5111333065177269
0.510790639802377
0.5105856487572474
0.5115682467058917
0.5110391223893895
0.5109418583219313
0.5102697910072671
0.5102783620524909
0.5107236524076734
0.5113858837326218
0.5094925069592676
0.5112832216576088
0.5102757399190742
0.5111543980364868
0.5109477252386868
0.5120870319588732
0.5121203424485319
0.511280277522792
0.5108090613572736
0.5120009200725297
0.512308211047862
0.5124820209123256
0.5113347811709098
0.5110548312746664
0.5108718559008559
0.5114695938472494
0.5110242377252027
0.5107191179105042
0.5115330400806679
0.5113161392543125
0.5117054637162928
0.5122242626351161
0.512480908000325
0.5111863735442138
0.5107104230185882
0.5118888934793469
0.5116177418229492
0.5117907940873019
0.5120997249636634
0.5122685519205203
0.5111914461339082
0.5117815286464861
0.5122408094073253
0.5121593353660805
0.5119175413545686
0.5117678978232947
0.5125127007274165
0.5116671387167676
0.511700679817508

list

In [ ]:
weights = weights_ref2023.copy()
weights[0] = 1.0
print('score:', 1.0-calc_score(weights))
weights[0] = 2.0
print('score:', 1.0-calc_score(weights))
weights[0] = 3.0
print('score:', 1.0-calc_score(weights))



0.47889410270516997
score: 0.47889410270516997
0.3976311884685055
score: 0.39763118846850554
0.3001425989147623
score: 0.30014259891476236


In [ ]:
scores  = df_test_repl.apply(lambda row: mutations[f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb'].loc[1], axis=1)
#weights = df_test_repl.apply(lambda row: mutations[f'/content/wildtypeA/wildtypeA_relaxed_{row.mut}_relaxed.pdb'].loc[0], axis=1)
weights = np.load('best_weights_5150.npy')

In [ ]:
scores_acc = np.zeros((scores.shape[0]), dtype=np.float32)
for i,prop in enumerate(props_all):  
  #scores_acc += weights[prop].loc[0]*rankdata(scores[prop])*-1.0
  scores_acc += weights[i]*rankdata(scores[prop])*-1.0
  #scores_acc += rankdata(scores[prop]*-1.0)

submit(scores_acc, 'submission.csv')
checkcsv('submission.csv')

val: 0.3467
sp: 0.9070 p: 0.9070 - 0.338
sp: 0.5008 p: 0.5008 - 0.368_constr
sp: 0.4825 p: 0.4825 - 0.472_roset
sp: 0.2705 p: 0.2705 - 0.482_p3
sp: 0.2894 p: 0.2894 - 0.488_term
sp: 0.4546 p: 0.4546 - 0.590
sp: 0.4130 p: 0.4130 - 0.603_del_spec
sp: 0.4052 p: 0.4052 - 0.613
sp: 0.4411 p: 0.4411 - 0.614
sp: 0.4776 p: 0.4776 - 0.615


In [ ]:
df = pd.read_csv('submission.csv')

ensamble(['368_constr','613'],[0.1,0.9],'submission_ens.csv')
checkcsv('submission_ens.csv')


0.1x368_constr
0.9x613
sp: 0.4501 - 0.338
sp: 0.6572 - 0.368_constr
sp: 0.7601 - 0.472_roset
sp: 0.7581 - 0.482_p3
sp: 0.7951 - 0.488_term
sp: 0.9251 - 0.590
sp: 0.9966 - 0.613
sp: 0.9968 - 0.614
sp: 0.9948 - 0.615
